In [1]:
base_path = "output/model"
constraints = {
    "Baseline": "baseline_1",
    "Positive": "Sentiment/Positive/constrained_eps_32_batch_8",
    "Negative": "Sentiment/Negative/constrained_eps_32_batch_8",
    "Neutral": "Sentiment/Neutral/constrained_eps_32_batch_8",
    "Formal": "Formality/Formal/constrained_eps_32_batch_8",
    "Informal": "Formality/Informal/constrained_eps_32_batch_8",
    "English": "Language/English/constrained_eps_32_batch_8",
    "French": "Language/French/constrained_eps_32_batch_8",
    "Spanish": "Language/Spanish/constrained_eps_32_batch_8",
    "Left": "Politics/Left/constrained_eps_32_batch_8",
    "Right": "Politics/Right/constrained_eps_32_batch_8",
    "Spam": "Attack/Spam/constrained_eps_32_batch_8",
    "Injection": "Attack/Injection/constrained_eps_32_batch_8"
}

file_dict = {}
for key, constraint in constraints.items():
    # First add paths with format base_path/i/constraint/
    paths = [f"{base_path}/{i}/{constraint}/" for i in range(0,5)]
    # Then add paths with format base_path/coco_i/constraint/
    paths.extend([f"{base_path}/coco_{i}/{constraint}/" for i in range(1,11)])
    file_dict[key] = paths

model_name="minigpt4"
if model_name == 'minigpt4':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", model_name) for path in paths]
elif model_name == 'llava':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", model_name) for path in paths]
elif model_name == 'blip':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", model_name) for path in paths]

# Similarity between oringinal and perturbed image
## LLaVA

In [ ]:
import argparse
from PIL import Image
import torch
import matplotlib.pyplot as plt

def parse_args():
    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--model_path", type=str, default="./ckpts/llava_llama_2_13b_chat_freeze")
    parser.add_argument("--gpu_id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--model_base", type=str, default=None)
    args = parser.parse_args(args=['--gpu_id', '0'])
    return args

# ========================================
#             Model Initialization
# ========================================
print('>>> Initializing Models')
from llava_llama_2.utils import get_model
args = parse_args()
print('model = ', args.model_path)
tokenizer, model, image_processor, model_name = get_model(args)
model.eval()
print('[Initialization Finished]\n')

## Minigpt4

In [ ]:
import argparse
import random
import numpy as np
import torch
import torch.backends.cudnn as cudnn
# import gradio as gr
from PIL import Image

from minigpt4.common.config import Config
from minigpt4.common.dist_utils import get_rank
from minigpt4.common.registry import registry

# imports modules for registration
from minigpt4.datasets.builders import *
from minigpt4.models import *
from minigpt4.processors import *
from minigpt4.runners import *
from minigpt4.tasks import *

from minigpt_utils import prompt_wrapper, generator

def parse_args():
    parser = argparse.ArgumentParser(description="Demo")

    parser.add_argument("--cfg-path", default="eval_configs/minigpt4_eval.yaml", help="path to configuration file.")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--mode", type=str, default='VisualChatBot',
                        choices=[ "TextOnly", "VisualChatBot" ],
                        help="Inference Mode: TextOnly: Text model only (Vicuna) \n VisualChatBot: Vision model + Text model (MiniGPT4) ")
    parser.add_argument("--image_file", type=str, default='./image.bmp',
                        help="Image file")
    parser.add_argument("--output_file", type=str, default='./result.jsonl',
                        help="Output file.")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )
    args = parser.parse_args(args=['--cfg-path', 'eval_configs/minigpt4_eval.yaml', 
                               '--gpu-id', '0', '--mode', 'VisualChatBot', '--image_file', 'clean_images/0.png', '--output_file', './result.jsonl'])
    # args = parser.parse_args()
    return args

def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True

# ========================================
#             Model Initialization
# ========================================

print('>>> Initializing Models')
args = parse_args()
cfg = Config(args)
model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))
vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
print('Initialization Finished')
my_generator = generator.Generator(model=model)

## BLIP

In [ ]:
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from PIL import Image
import json



def parse_args():

    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--data_path", type=str, default="instruction_data/0/Sentiment/dataset.csv")
    parser.add_argument("--image_file", type=str, default='./image.bmp',
                        help="Image file")
    parser.add_argument("--output_file", type=str, default='./result.jsonl',
                        help="Output file.")
    parser.add_argument("--instruction", type=str, default=None,
                    choices=[ "positive", "negative", "neutral", "irony", "non_irony", "formal", "informal", "french", "english", "spanish", "left","right","inference_content_evaluation","injection","spam"],
                        help="Instruction to be used for the attack.")
    parser.add_argument("--image_index", type=int, default=0)
    # args = parser.parse_args()
    args = parser.parse_args(args=['--data_path', 'instruction_data/0/Attack/dataset.csv', 
                                '--image_file', 'clean_images/0.png', '--output_file', './result.jsonl'])
    return args


# ========================================
#             Model Initialization
# ========================================

print('>>> Initializing Models')

from lavis.models import load_model_and_preprocess

args = parse_args()
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

# remember to modify the parameter llm_model in ./lavis/configs/models/blip2/blip2_instruct_vicuna13b.yaml to the path that store the vicuna weights
model, vis_processor, _ = load_model_and_preprocess(
        name='blip2_vicuna_instruct',
        model_type='vicuna13b',
        is_eval=True,
        device=device,
    )
model.eval()
"""
Source code of the model in:
    ./lavis/models/blip2_models/blip2_vicuna_instruct.py
"""


img = Image.open(args.image_file).convert('RGB')
img = vis_processor["eval"](img).unsqueeze(0).to(device)


In [4]:
from PIL import Image
from pytorch_msssim import ssim
from torchvision import transforms
import torch
import matplotlib.pyplot as plt

def denormalize(images):
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).cuda()
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).cuda()
    images = images * std[None, :, None, None]
    images = images + mean[None, :, None, None]
    return images

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return image

def calculate_similarity(image1, image2, metric, model):
    if metric=='llava':
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        if not isinstance(image1, torch.Tensor):
            image1 = image_processor.preprocess(image1, return_tensors='pt')['pixel_values'].cuda()
            image2 = image_processor.preprocess(image2, return_tensors='pt')['pixel_values'].cuda()
        emb_image1 = model.encode_images(image1.half())
        emb_image2 = model.encode_images(image2.half())
        cos_sim = cos(emb_image1.view(-1).to(torch.float32), emb_image2.view(-1).to(torch.float32))
        return cos_sim
    elif metric=='minigpt4':
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        if not isinstance(image1, torch.Tensor):
            image1 = [vis_processor(image1).unsqueeze(0).to(model.device)]
            image2 = [vis_processor(image2).unsqueeze(0).to(model.device)]
        else:
            image1=[image1]
            image2=[image2]
        prompt1 = prompt_wrapper.Prompt(model=model, img_prompts=[image1])
        emb_image1 = prompt1.img_embs[0][0]
        prompt2 = prompt_wrapper.Prompt(model=model, img_prompts=[image2])
        emb_image2 = prompt2.img_embs[0][0]
        cos_sim = cos(emb_image1.view(-1).to(torch.float32), emb_image2.view(-1).to(torch.float32))
        return cos_sim
    elif metric=='blip':
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        if not isinstance(image1, torch.Tensor):
            image1 = vis_processor["eval"](image1).unsqueeze(0).to(device)
            image2 = vis_processor["eval"](image2).unsqueeze(0).to(device)
        with model.maybe_autocast():
            emb_image1 = model.ln_vision(model.visual_encoder(image1))
            emb_image2 = model.ln_vision(model.visual_encoder(image2))
        cos_sim = cos(emb_image1.view(-1).to(torch.float32), emb_image2.view(-1).to(torch.float32))
        return cos_sim

def calculate_transformations_sim(image, metric, model, show_plots=True):

    transformations = [
        transforms.GaussianBlur(kernel_size=9, sigma=(1.0, 5.0)),
        transforms.RandomAffine(degrees=45),  # Rotate by 45 degrees
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),
        transforms.RandomHorizontalFlip(p=1),
        transforms.RandomPerspective(distortion_scale=0.5, p=1)
    ]
    
    # image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].cuda()
    # for blip
    image_tensor = vis_processor["eval"](image).unsqueeze(0).to(device)
    if show_plots:
        fig, axs = plt.subplots(1, len(transformations) + 1, figsize=(15, 10))  # +1 for the original image
        axs[0].imshow(denormalize(image_tensor)[0].detach().cpu().permute(1, 2, 0))
        axs[0].set_title('Original')
        axs[0].axis('off')

    sim_list = []
    for idx, transform in enumerate(transformations):
        transformed_image_tensor = transform(image_tensor)  # Apply transformation
        if show_plots:
            axs[idx + 1].imshow(denormalize(transformed_image_tensor)[0].detach().cpu().permute(1, 2, 0))
            axs[idx + 1].set_title(transform.__class__.__name__)
            axs[idx + 1].axis('off')
        if metric=='ssim':
            sim = calculate_similarity(denormalize(image_tensor), denormalize(transformed_image_tensor),model, metric)
        else:
            sim = calculate_similarity(denormalize(image_tensor), denormalize(transformed_image_tensor),model, metric)
        sim_list.append(sim)
    if show_plots:
        plt.show()
        
    return sum(sim_list)/len(sim_list)

In [ ]:
from torchvision.utils import save_image
sim_dict={}
sim_metric='llava'
temp_list=[]

def jpeg_transform(X):
    image_filename = 'dummy.jpg'
    save_image(torch.squeeze(denormalize(X)), image_filename)
    image = load_image(image_filename)
    if sim_metric=='blip':
        jpeg_X = vis_processor["eval"](image).unsqueeze(0).to(device)
    if sim_metric == 'llava' or sim_metric == 'ssim':
        jpeg_X = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].cuda()
    else:
        jpeg_X = vis_processor(image).unsqueeze(0).to(model.device)
    jpeg_X=jpeg_X.clamp(0,1)
    return jpeg_X.cpu()

# Define a list of transformations to apply
transformations = [
    jpeg_transform,
    transforms.GaussianBlur(kernel_size=9, sigma=(1.0, 5.0)),
    transforms.RandomAffine(degrees=45),  # Rotate by 45 degrees
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5)
]


for idx, transform in enumerate(transformations):
    print(transform)
    temp_list = []
    for key in file_dict:
        for file in file_dict[key]:
            image_file1 = file + "bad_prompt.bmp"
            image1 = load_image(image_file1)
            if sim_metric=='blip':
                jpeg_X = vis_processor["eval"](image1).unsqueeze(0).to(device)
            if sim_metric == 'llava' or sim_metric == 'ssim':
                image1_tensor = image_processor.preprocess(image1, return_tensors='pt')['pixel_values'].cuda()
            else:
                image1_tensor = vis_processor(image1).unsqueeze(0).to(model.device)
            transformed_image_tensor = transform(image1_tensor).cuda()
            sim = calculate_similarity(image1_tensor, transformed_image_tensor, sim_metric, model)
            temp_list.append(sim)
    mean_sim = torch.mean(torch.stack(temp_list)).item()
    std_sim = torch.std(torch.stack(temp_list)).item()
    sim_dict[transform] = {'mean': round(mean_sim, 3), 'std': round(std_sim, 3)}

# Print the final dictionary with mean and standard deviation values
print(sim_dict)

In [ ]:
file_list=[
    "clean_images/coco_1.jpg",
    "clean_images/coco_2.jpg",
    "clean_images/coco_3.jpg",
    "clean_images/coco_4.jpg",
    "clean_images/coco_5.jpg",
    "clean_images/coco_6.jpg",
    "clean_images/coco_7.jpg",
    "clean_images/coco_8.jpg",
    "clean_images/coco_9.jpg",
    "clean_images/coco_10.jpg"
]

for idx, transform in enumerate(transformations):
    print(transform)
    temp_list = []
    for file in file_list:
        image_file1 = file
        image1 = load_image(image_file1)
        if sim_metric=='blip':
            image1_tensor = vis_processor["eval"](image1).unsqueeze(0).to(device)
        if sim_metric == 'llava' or sim_metric == 'ssim':
            image1_tensor = image_processor.preprocess(image1, return_tensors='pt')['pixel_values'].cuda()
        else:
            image1_tensor = vis_processor(image1).unsqueeze(0).to(model.device)
        transformed_image_tensor = transform(image1_tensor).cuda()
        sim = calculate_similarity(image1_tensor, transformed_image_tensor, sim_metric, model)
        temp_list.append(sim)
    mean_sim = torch.mean(torch.stack(temp_list)).item()
    std_sim = torch.std(torch.stack(temp_list)).item()
    sim_dict[transform] = {'mean': round(mean_sim, 3), 'std': round(std_sim, 3)}

# Print the final dictionary with mean and standard deviation values
print(sim_dict)